<a href="https://colab.research.google.com/github/dineshRaja29/TextEncodings/blob/main/textEncodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#<font color = 'green'><b>INTRODUCTION


We will explore various ways to encode text (conversion of text into numeric vector/scaler) from classical to state of art approach. There are broadly four methods which are extensively used in ML/DL.



- BoW using SkLearn + Sparse Matrices
- TF-IDF using SkLearn
- Word2Vec using Gensim
- BERT using bert-serving library + APIs.




#<font color = 'green'><b>Data loading

In [ ]:
import shutil
import os
import sys
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
# pre-requisted for stopword and word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# pre-requisted for Lemmatize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Source: http://ai.stanford.edu/~amaas/data/sentiment/
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2022-05-03 12:48:26--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  15.8MB/s    in 13s     

2022-05-03 12:48:39 (6.35 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
# uncompress and see the data
! ls
! pwd

aclImdb_v1.tar.gz  sample_data
/content


In [ ]:
! tar -zxvf aclImdb_v1.tar.gz >& /dev/null

In [ ]:
! ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
#Google: "Unzip tar gz file colab" ----> https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook
# shutil.unpack_archive("/content/aclImdb_v1.tar.gz", "/content/")
# ! ls /content/

In [ ]:
! ls -l /content/aclImdb

total 1724
-rw-r--r-- 1 7297 1000 903029 Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 845980 Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000   4037 Jun 26  2011 README
drwxr-xr-x 4 7297 1000   4096 May  3 12:48 test
drwxr-xr-x 5 7297 1000   4096 May  3 12:48 train


In [ ]:
! head -10 /content/aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [ ]:
! ls /content/aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [ ]:
! ls /content/aclImdb/train/pos | head -10

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt


In [ ]:
! cat /content/aclImdb/train/pos/6250_10.txt

There is something special about the Austrian movies not only by Seidl, but by Spielmann and other directors as well. This is the piercing sense of reality that never leaves the viewer throughout the movie. Hundstage is no exception. This effect is achieved not only by the depicted stories but also by actors playing. In Hundstage I have never had the feeling that these are actors playing, but real people instead. So real is the visceral feeling of the viewer...Almost as if the grumpy pensioner or lonely lady in the movie are living below you in your block.<br /><br />Any person living in Vienna can without any doubt painfully recognize the people in the movie with their meckern/sudern (complaining), their hidden sexual urges and the prolo macho guys. This is further reinforced by the Viennese dialect which is, according to many, especially made for complaining as a way of life. A special parochialism and arrogance typical for Vienna are also very well portrayed.<br /><br />The Viennese

In [ ]:
# load data from k-reviews from imdb/train/pos reviews

k=100

raw_data = [] # empty list, 


index_file = dict(); # store mapping from index to filename

directory = r'/content/aclImdb/train/pos/'

i=0

for f in os.listdir(directory): # for each file in the subfolder
      
  if f.endswith(".txt"): # check for text file
    fname = directory + "/" + f
    
    tmp = open(fname, "r") # read file 

    raw_data.append(tmp.read())
    index_file[i] = fname
        
    i += 1

    if i==k: # read k files
      break


In [ ]:
print(i)
print(index_file)
print(raw_data[0])

100
{0: '/content/aclImdb/train/pos//11958_9.txt', 1: '/content/aclImdb/train/pos//10774_8.txt', 2: '/content/aclImdb/train/pos//7436_10.txt', 3: '/content/aclImdb/train/pos//1486_7.txt', 4: '/content/aclImdb/train/pos//6649_8.txt', 5: '/content/aclImdb/train/pos//3264_8.txt', 6: '/content/aclImdb/train/pos//4646_7.txt', 7: '/content/aclImdb/train/pos//7140_10.txt', 8: '/content/aclImdb/train/pos//197_9.txt', 9: '/content/aclImdb/train/pos//9939_10.txt', 10: '/content/aclImdb/train/pos//2286_10.txt', 11: '/content/aclImdb/train/pos//4739_7.txt', 12: '/content/aclImdb/train/pos//11590_10.txt', 13: '/content/aclImdb/train/pos//3431_10.txt', 14: '/content/aclImdb/train/pos//10787_10.txt', 15: '/content/aclImdb/train/pos//5397_9.txt', 16: '/content/aclImdb/train/pos//8633_8.txt', 17: '/content/aclImdb/train/pos//7609_10.txt', 18: '/content/aclImdb/train/pos//6953_7.txt', 19: '/content/aclImdb/train/pos//8643_7.txt', 20: '/content/aclImdb/train/pos//10456_10.txt', 21: '/content/aclImdb/trai

##<font color = 'green'><b> Data preprocessing

Not always removing stopword is helpful, 'not' is considered as a stopword
for example:
- He is bad boy
- He is not a bad boy <br>
 After removal of stopword both makes the same sentences thus, harmful here <br>

 Another approach to fix this is to filter the list of stopwords based on language which you are working


In [ ]:

example_sent = "This is a sample sentence, showing off the stop words filtration."

def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += " " + w
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 
print(stop_words)

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function

{'itself', 'but', "shan't", "couldn't", "you're", 'myself', "didn't", 'few', 'himself', 'not', 'or', 'their', 'shouldn', 'how', 'both', "aren't", 'because', 'more', 'he', 'should', 'does', 'all', 'why', "don't", 'before', 'doing', "isn't", 'which', "haven't", "hadn't", 'm', 'haven', 'his', "it's", 'until', 'too', "wasn't", 'now', 'was', 'further', 'its', 'here', "mightn't", 'll', "doesn't", 'hadn', "shouldn't", 'where', 'whom', 'only', 't', 'own', 'what', 'than', 'off', "you'll", "she's", 'won', 'didn', 'is', 'very', 'had', 'a', 'during', 'are', 'hers', "wouldn't", 'doesn', "won't", 'wouldn', 'up', 'being', 'about', 'd', 'wasn', 'yourself', 'who', 'it', 'mustn', 'y', "you'd", 'into', 'can', 'couldn', 'with', 'we', 'did', 'as', 'each', 'that', 'be', 'out', 'my', 're', 'do', 'over', 'some', 's', 'those', "should've", "hasn't", 'they', 'you', 'once', 'any', 'your', 'her', 'on', 'at', 'just', 'isn', 'having', 'through', 'the', 'these', 'will', 'yourselves', 'an', 'weren', 'to', 'o', 'while

In [ ]:
# Lemmatize
# Source: https://pythonprogramming.net/lemmatizing-nltk-tutorial/

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

cat
cactus
goose
rock
python
good
best
run
run


In [ ]:

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWord_Lemmatize(sent, stop_words, lemmatizer ):
  word_tokens = word_tokenize(sent) # tokenize
  return_sent = "";
  
  for w in word_tokens:
      if w not in stop_words:
          return_sent +=  " " + lemmatizer.lemmatize(w) # lemmatize w before adding it to the return_sent
  return return_sent

stop_words = set(stopwords.words('english')) # NLTK 
lemmatizer = WordNetLemmatizer()

print(raw_data[0])
print(stopWord_Lemmatize(raw_data[0], stop_words, lemmatizer) ) # call the function


First of all, let me comment that the audience LOVED it from the first moment. Perhaps current events in the Middle-East led people to take the attitude, "I came for a comedy and by George I'm going to enjoy it." but for whatever reason, everybody seemed really into the comedy of it. The last few times Woody has tried to do a straight comedy (Small Time Crooks, Curse of the Jade Scorpion, Hollywood Ending) I've felt like the one-liners felt strained and a bit antiquated. I remember thinking at one point, "That would have been funny in the early sixties." So going in to this movie, I was afraid Woody was becoming tone deaf, however, in this one his comic sensibilities were in perfect tune. Admittedly, there were plenty of my fellow AARP card carrying folks in the screening, but there were also plenty of 20-somethings and 30-somethings as well, and they all seemed to get it and give up the occasional belly laugh in addition to numerous guffaws, chuckles and the like. In many instances, t

In [ ]:

# pre-process the k sentences and store the result
processed_data = []

for sent in raw_data:
  processed_data.append(stopWord_Lemmatize(sent, stop_words, lemmatizer) )

#<font color = 'green'><b>Bag-of-words (BoW)

In [ ]:
# Google "sklearn bag of words" ---> https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X_BoW = count_vect.fit_transform(processed_data)

print(count_vect.get_feature_names()) 

['01', '02', '04', '10', '100', '11', '12', '14', '15', '17', '1794', '1798', '1928', '1929', '1931', '1939', '1940', '1943', '1946', '1947', '1948', '1950', '1950s', '1954', '1958', '1969', '1972', '1973', '1979', '1980s', '1982', '1986', '1990', '1993', '1994', '1st', '20', '2001', '2002', '2004', '2005', '2007', '2008', '20s', '20yrs', '21', '216', '21st', '25', '30', '300', '31', '3rd', '40', '40s', '50', '56', '69', '70', '76', '80', '85', '90', '95', 'aarp', 'abandoned', 'abbas', 'ability', 'able', 'abound', 'about', 'above', 'abovementioned', 'abruptly', 'absent', 'absolute', 'absolutely', 'academic', 'academy', 'accent', 'accepts', 'accessible', 'acclaim', 'accompanied', 'accomplished', 'according', 'account', 'accumulate', 'accuracy', 'achieve', 'achievement', 'acosta', 'acquire', 'acquired', 'across', 'act', 'acted', 'acting', 'action', 'active', 'activism', 'activist', 'actor', 'actors', 'actress', 'actual', 'actually', 'adam', 'adapt', 'adaptation', 'adapted', 'added', 'add

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(type(X_BoW))

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
# Sparse representations vs dense Matrices

#Refer: https://docs.scipy.org/doc/scipy/reference/sparse.html


print(X_BoW.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

# Convert X_BoW to dense
# Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.todense.html#scipy.sparse.csr_matrix.todense
X_BoW_Dense = X_BoW.todense();
print(X_BoW_Dense.data.nbytes)

print(X_BoW.shape)
print(X_BoW_Dense.shape)
# Always use sparse-matrix in these situations

84040
3660000
(100, 4575)
(100, 4575)


In [ ]:
print(X_BoW[0,:])

  (0, 1579)	2
  (0, 2365)	1
  (0, 842)	1
  (0, 332)	3
  (0, 2442)	1
  (0, 2649)	1
  (0, 2988)	2
  (0, 989)	2
  (0, 1398)	1
  (0, 2599)	1
  (0, 1271)	1
  (0, 2352)	1
  (0, 2974)	2
  (0, 4003)	1
  (0, 327)	1
  (0, 632)	1
  (0, 834)	4
  (0, 1718)	1
  (0, 1766)	3
  (0, 1351)	1
  (0, 4448)	1
  (0, 3264)	1
  (0, 1403)	1
  (0, 3541)	2
  (0, 3262)	1
  :	:
  (0, 4544)	1
  (0, 156)	1
  (0, 783)	1
  (0, 1276)	1
  (0, 3188)	1
  (0, 627)	1
  (0, 4086)	1
  (0, 234)	1
  (0, 2550)	1
  (0, 456)	1
  (0, 34)	1
  (0, 603)	1
  (0, 2884)	1
  (0, 2363)	3
  (0, 3908)	1
  (0, 2603)	1
  (0, 250)	1
  (0, 683)	1
  (0, 1061)	1
  (0, 1887)	1
  (0, 1621)	1
  (0, 1566)	1
  (0, 1637)	1
  (0, 47)	1
  (0, 699)	1


In [ ]:
print(X_BoW[0,818])
print(X_BoW[0,817])

0
0


In [ ]:
print(X_BoW_Dense[0,:])
print(X_BoW_Dense[0,818])
print(X_BoW_Dense[0,817])

[[0 0 0 ... 0 0 0]]
0
0


#<font color = 'green'><b>TD-IDF

In [ ]:
# Google: "TF-IDF SkLearn" ---> https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(raw_data)

print(vectorizer.get_feature_names())

['01', '01 enterprise', '02', '02 sam', '04', '04 like', '10', '10 31', '10 dvd', '10 grisham', '10 minute', '10 minutes', '10 respectively', '10 seen', '10 truly', '100', '100 metacritic', '11', '11 1973', '11 br', '11 countries', '11 minute', '12', '12 02', '12 minute', '12 year', '14', '14 time', '14 vance', '15', '15 minute', '15 minutes', '17', '17 years', '1794', '1794 95', '1798', '1798 world', '1928', '1928 davies', '1929', '1929 1947', '1931', '1931 supposedly', '1939', '1939 fringe', '1940', '1940 just', '1943', '1943 nowadays', '1946', '1946 rabbit', '1947', '1947 story', '1948', '1948 great', '1950', '1950 emphasize', '1950 shown', '1950 stands', '1950s', '1950s forgot', '1950s represented', '1954', '1954 just', '1958', '1969', '1969 cancer', '1972', '1972 persuaded', '1973', '1973 depicted', '1979', '1979 gregory', '1980s', '1980s best', '1982', '1982 merely', '1986', '1986 comments', '1986 covered', '1986 inferior', '1990', '1990 world', '1993', '1993 worth', '1994', '199

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(type(X_tfidf))
print(X_tfidf.shape)
print(X_tfidf.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

X_tfidf_dense = X_tfidf.todense()
print(type(X_tfidf_dense))
print(X_tfidf_dense.shape)
print(X_tfidf_dense.data.nbytes)

<class 'scipy.sparse.csr.csr_matrix'>
(100, 15258)
160088
<class 'numpy.matrix'>
(100, 15258)
12206400


#<font color = 'green'><b>Word2Vec

In [ ]:
# Gensim
# Refer: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

# Download pretrained  vectors
# https://www.quora.com/How-can-I-download-the-Google-news-word2vec-pretrained-model-from-a-Ubuntu-terminal
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
! gunzip GoogleNews-vectors-negative300.bin.gz


--2022-05-03 12:59:19--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.196.128
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.196.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-03 12:59:19 ERROR 404: Not Found.



In [ ]:
! ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
# takes time as the model is large to laod into RAM
# RAM consumption also shoots up. Kernel could restart and give you a larger RAM isnatnce like 25GB RAM

from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
v = model['queen'];
print(type(v))
print(v.shape)

#<font color = 'green'><b>BERT


- BERT vs Word2Vec
- BERT: Contextual encodings

In [1]:
# There are many ways to obatin BERT encodings. We are using one of the 
# most simple+popular approaches

# In DL chpaters, we will see how to do it in Keras.

# use GPU based instance: Runtime---> Change Runtime --> GPU

# Chaneg to TensorFlow Version 1.x 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#https://github.com/hanxiao/bert-as-service

# https://github.com/hanxiao/bert-as-service/issues/380
# Install BERT-SERVING client and Server
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 61 kB 242 kB/s 
     |████████████████████████████████| 357 kB 11.0 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=9e38d59d974289fbd25ce81322f5a854d00e47f8b08b13e677479ebca94878a9
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [3]:
# dowload pretrained models
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2022-05-03 17:53:35--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 108.177.112.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   167MB/s    in 2.3s    

2022-05-03 17:53:38 (167 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [4]:
# Start BERT_SERVER on the current computer
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
# Use bert-client from python
# Takes time to execute as it uses GPU
from bert_serving.client import BertClient
bc = BertClient()
print (bc.encode(['First do it', 'then do it right', 'then do it better'])) # list of setences

In [ ]:
v = bc.encode([raw_data[0]])

In [ ]:
print(type(v))
print(v.shape)